# Loading and preprocesing

Importing libraries

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [21]:
root_dir="/content/drive/MyDrive/Thesis resources/Datasets/dataset'/Mini_DDSM_Upload"
number_of_images={}
for dir in os.listdir(root_dir):
  number_of_images[dir]=len(os.listdir(os.path.join(root_dir,dir)))

Creating Traning Folder

In [22]:

def dataFolder(p,split):
  if not os.path.exists("/content/drive/MyDrive/"+p):
    os.mkdir("/content/drive/MyDrive/"+p)
    for dir in os.listdir(root_dir):
      os.makedirs("/content/drive/MyDrive/"+p+"/"+dir)
      for img in np.random.choice(a=os.listdir(os.path.join(root_dir,dir)),size=(math.floor(split*number_of_images[dir])-5),replace=False):
        O=os.path.join(root_dir,dir,img)
        D=os.path.join("/content/drive/MyDrive/"+p,dir)
        shutil.copy(O,D)
        os.remove(O)
  else:
    print(f"{p} Folder exist")

In [ ]:
dataFolder("thesis_train",0.7)
dataFolder("thesis_test",0.15)
dataFolder("thesis_val",0.15)

In [ ]:
root_dir='/content/drive/MyDrive/Thesis work/DataSet/Mini_DDSM'
number_of_images={}
for dir in os.listdir(root_dir):
  number_of_images[dir]=len(os.listdir(os.path.join(root_dir,dir)))
number_of_images.items()